In [2]:
%pip install tqdm Pillow ipywidgets jupyter ipykernel

  Using cached nbformat-5.10.4-py3-none-any.whl.metadata (3.6 kB)
  Using cached jsonschema-4.23.0-py3-none-any.whl.metadata (7.9 kB)
  Using cached fastjsonschema-2.21.1-py3-none-any.whl.metadata (2.2 kB)
  Using cached jsonschema_specifications-2024.10.1-py3-none-any.whl.metadata (3.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 55.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 113.8 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.5/258.5 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 121.7 MB/s eta 0:00:0000:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.4/163.4 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/7

In [17]:
import os
import pathlib
from PIL import Image
import multiprocessing
from functools import partial
import concurrent.futures
from tqdm.notebook import tqdm  # Using notebook version of tqdm

# Set these variables to your desired paths and size
parent_dir = '/Volumes/External SSD/rxrx3/'
output_dir = '/Volumes/External SSD/rxrx3_resized/'
target_size = (512, 512)  # Change this if you want a different size

# Function to resize a single image
def resize_image(src_path, dst_path):
    try:
        # Create destination directory if needed
        os.makedirs(os.path.dirname(dst_path), exist_ok=True)

        # Open, resize and save the image
        with Image.open(src_path) as img:
            # Resize with antialiasing
            resized_img = img.resize(target_size, Image.LANCZOS)
            # Save the resized image
            resized_img.save(dst_path)
        return True
    except Exception as e:
        return (src_path, str(e))

# Get all image paths from a directory
def get_image_paths(source_dir, dest_dir):
    image_paths = []

    # Get all directories first
    all_dirs = list(os.walk(source_dir))

    # Display progress for directory scanning
    for root, _, files in tqdm(all_dirs, desc="Scanning directories"):
        rel_path = os.path.relpath(root, source_dir)

        for file in files:
            if file.lower().endswith((".png", ".jpg", ".jpeg", ".tif", ".tiff")):
                src_path = os.path.join(root, file)

                # Construct destination path
                if rel_path == ".":
                    dst_path = os.path.join(dest_dir, file)
                else:
                    dst_path = os.path.join(dest_dir, rel_path, file)

                image_paths.append((src_path, dst_path))

    return image_paths

# Process a single directory
def process_directory(input_dir, output_base_dir):
    """Process a single directory of images"""
    
    # Get directory name and create output dir name with _512 suffix
    dir_name = os.path.basename(input_dir)
    output_dir = os.path.join(output_base_dir, f"{dir_name}_512")
    
    # Create output directory
    os.makedirs(output_dir, exist_ok=True)
    
    print(f"\nProcessing directory: {dir_name}")
    print(f"Output directory: {output_dir}")
    
    # Get all image paths
    image_paths = get_image_paths(input_dir, output_dir)
    print(f"Found {len(image_paths)} images to resize")
    
    if not image_paths:
        print(f"No images found in {input_dir}. Skipping...")
        return (0, 0)
    
    # Get number of CPU cores
    num_cores = multiprocessing.cpu_count()
    # Use 80% of available cores to avoid overloading the system
    num_workers = max(1, int(num_cores * 0.8))
    print(f"Using {num_workers} workers")
    
    # Process images in parallel
    results = []
    failures = []

    # Process in batches to show progress
    batch_size = 100  # Adjust based on memory constraints
    total_batches = (len(image_paths) + batch_size - 1) // batch_size

    for i in range(0, len(image_paths), batch_size):
        batch = image_paths[i:i + batch_size]
        batch_num = i // batch_size + 1
        print(f"Processing batch {batch_num}/{total_batches}")

        # Process current batch
        with concurrent.futures.ThreadPoolExecutor(max_workers=num_workers) as executor:
            # Map function directly to all items in batch
            batch_src_paths, batch_dst_paths = zip(*batch)
            for idx, result in enumerate(
                tqdm(
                    executor.map(resize_image, batch_src_paths, batch_dst_paths),
                    total=len(batch),
                    desc=f"Resizing images ({dir_name})"
                )
            ):
                if result is True:
                    results.append(result)
                else:
                    failures.append(result)
    
    print(f"\nResizing complete for {dir_name}!")
    print(f"Successfully resized: {len(results)}/{len(image_paths)} images")
    
    if failures:
        print(f"Failed to resize: {len(failures)} images")
        print("First few errors:")
        for src_path, error in failures[:5]:
            print(f"- {src_path}: {error}")

        if len(failures) > 5:
            print(f"... and {len(failures) - 5} more errors")
            
    return (len(results), len(failures))

# Main execution code
# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Get subdirectories
subdirs = [os.path.join(parent_dir, d) for d in os.listdir(parent_dir) 
          if os.path.isdir(os.path.join(parent_dir, d))]

if not subdirs:
    print(f"No subdirectories found in {parent_dir}")
else:
    print(f"Found {len(subdirs)} subdirectories to process:")
    for d in subdirs:
        print(f"- {os.path.basename(d)}")
    print("-" * 50)
    
    # Process each subdirectory
    total_success = 0
    total_failures = 0
    
    for subdir in subdirs:
        success, failures = process_directory(subdir, output_dir)
        total_success += success
        total_failures += failures
    
    # Final statistics
    print("\n" + "="*50)
    print("OVERALL STATISTICS:")
    print(f"Processed {len(subdirs)} directories")
    print(f"Successfully resized: {total_success} images")
    print(f"Failed: {total_failures} images")
    print("="*50)

Found 2 subdirectories to process:
- rxrx3
- rxrx3_temp
--------------------------------------------------

Processing directory: rxrx3
Output directory: /Volumes/External SSD/rxrx3_resized/rxrx3_512


Scanning directories:   0%|          | 0/30 [00:00<?, ?it/s]

Found 155304 images to resize
Using 12 workers
Processing batch 1/1554


Resizing images (rxrx3):   0%|          | 0/100 [00:00<?, ?it/s]

Processing batch 2/1554


...

Processing batch 28/29


Resizing images (rxrx3_temp):   0%|          | 0/100 [00:00<?, ?it/s]

Processing batch 29/29


Resizing images (rxrx3_temp):   0%|          | 0/29 [00:00<?, ?it/s]


Resizing complete for rxrx3_temp!
Successfully resized: 2829/2829 images

OVERALL STATISTICS:
Processed 2 directories
Successfully resized: 158133 images
Failed: 0 images


In [18]:
import os
import shutil
from pathlib import Path
import glob

# Define source and destination directories
source_dir = '/Volumes/External SSD/cp_1_gene_1/'
destination_dir = '/Volumes/External SSD/cp_1_gene_1_mixed/'

# Create destination directory if it doesn't exist
os.makedirs(destination_dir, exist_ok=True)

# Get all subdirectories in the source directory
subdirs = [d for d in os.listdir(source_dir) if os.path.isdir(os.path.join(source_dir, d)) and d != 'mixed_images']

# Common image file extensions
image_extensions = ['*.jpg', '*.jpeg', '*.png', '*.tif', '*.tiff']

# Counter for tracking progress
copied_files = 0
total_files = 0

# Function to copy images from a directory
def copy_images_from_dir(dir_path, subdir_name):
    global copied_files, total_files
    
    # Get all image files in the directory
    image_files = []
    for ext in image_extensions:
        image_files.extend(glob.glob(os.path.join(dir_path, ext)))
        image_files.extend(glob.glob(os.path.join(dir_path, '**', ext), recursive=True))
    
    total_files += len(image_files)
    
    # Copy each image to the destination directory
    for img_path in image_files:
        # Get the original filename
        filename = os.path.basename(img_path)
        
        # Add the folder name as a prefix to the original filename
        new_filename = f"{subdir_name}_{filename}"
        dest_path = os.path.join(destination_dir, new_filename)
        
        # Copy the file
        shutil.copy2(img_path, dest_path)
        copied_files += 1
        
        # Print progress every 100 files
        if copied_files % 100 == 0:
            print(f"Copied {copied_files} files so far...")

# Process each subdirectory
print(f"Found {len(subdirs)} subdirectories to process.")
for subdir in subdirs:
    subdir_path = os.path.join(source_dir, subdir)
    print(f"Processing directory: {subdir}")
    copy_images_from_dir(subdir_path, subdir)

print(f"Copying complete! Copied {copied_files} image files to {destination_dir}")
print(f"Total files processed: {total_files}")

Found 22 subdirectories to process.
Processing directory: compound-001_plate10
Copied 100 files so far...
Copied 200 files so far...
Copied 300 files so far...
Copied 400 files so far...
Copied 500 files so far...
Copied 600 files so far...
Copied 700 files so far...
Copied 800 files so far...
Copied 900 files so far...
Copied 1000 files so far...
Copied 1100 files so far...
Copied 1200 files so far...
Copied 1300 files so far...
Copied 1400 files so far...
Copied 1500 files so far...
Copied 1600 files so far...
Copied 1700 files so far...
Copied 1800 files so far...
Copied 1900 files so far...
Copied 2000 files so far...
Copied 2100 files so far...
Copied 2200 files so far...
Copied 2300 files so far...
Copied 2400 files so far...
Copied 2500 files so far...
Copied 2600 files so far...
Copied 2700 files so far...
Copied 2800 files so far...
Copied 2900 files so far...
Copied 3000 files so far...
Copied 3100 files so far...
Copied 3200 files so far...
Copied 3300 files so far...
Copied

In [ ]:
import os
import shutil
from pathlib import Path
import glob

# Define source and destination directories
source_dir = '/Volumes/External SSD/rxrx3_resized/rxrx3_512/'
destination_dir = '/Volumes/External SSD/rxrx3_resized/rxrx3_512/mixed_images/'

# Create destination directory if it doesn't exist
os.makedirs(destination_dir, exist_ok=True)

# Get all subdirectories in the source directory
subdirs = [d for d in os.listdir(source_dir) if os.path.isdir(os.path.join(source_dir, d)) and d != 'mixed_images']

# Common image file extensions
image_extensions = ['*.jpg', '*.jpeg', '*.png', '*.tif', '*.tiff']

# Counter for tracking progress
copied_files = 0
total_files = 0

# Function to copy images from a directory
def copy_images_from_dir(dir_path, subdir_name):
    global copied_files, total_files
    
    # Get all image files in the directory
    image_files = []
    for ext in image_extensions:
        image_files.extend(glob.glob(os.path.join(dir_path, ext)))
        image_files.extend(glob.glob(os.path.join(dir_path, '**', ext), recursive=True))
    
    total_files += len(image_files)
    
    # Copy each image to the destination directory
    for img_path in image_files:
        # Get the original filename
        filename = os.path.basename(img_path)
        
        # Add the folder name as a prefix to the original filename
        new_filename = f"{subdir_name}_{filename}"
        dest_path = os.path.join(destination_dir, new_filename)
        
        # Copy the file
        shutil.copy2(img_path, dest_path)
        copied_files += 1
        
        # Print progress every 100 files
        if copied_files % 100 == 0:
            print(f"Copied {copied_files} files so far...")

# Process each subdirectory
print(f"Found {len(subdirs)} subdirectories to process.")
for subdir in subdirs:
    subdir_path = os.path.join(source_dir, subdir)
    print(f"Processing directory: {subdir}")
    copy_images_from_dir(subdir_path, subdir)

print(f"Copying complete! Copied {copied_files} image files to {destination_dir}")
print(f"Total files processed: {total_files}")

In [10]:

!ls -1 /Volumes/External\ SSD/rxrx3/rxrx3/compound-001_plate10/ | wc -l

!ls -1 /Volumes/External\ SSD/rxrx3/rxrx3/gene-001_plate2/ | wc -l


    1860
    8214


In [5]:
!ls -1 /Volumes/External\ SSD/rxrx3/rxrx3/

compound-001_plate10
compound-001_plate11
compound-001_plate12
compound-001_plate13
compound-001_plate14
compound-001_plate15
compound-001_plate16
compound-001_plate17
compound-001_plate18
compound-002_plate1
compound-003_plate1
gene-001_plate2
gene-001_plate3
gene-001_plate4
gene-001_plate5
gene-002_plate1
gene-003_plate1
gene-004_plate1
gene-005_plate1
gene-006_plate1
gene-007_plate1
gene-008_plate1


In [14]:
!ls /Volumes/External SSD/rxrx3_resized/rxrx3_512/mixed_images/

ls: /Volumes/External: No such file or directory
ls: SSD/rxrx3_resized/rxrx3_512/mixed_images/: No such file or directory


In [42]:
import os
from collections import defaultdict

# Path to the mixed images directory
mixed_images_path = "/Volumes/External SSD/cp_1_gene_1_mixed/"

# Get list of all files in directory
files = os.listdir(mixed_images_path)

# Dictionary to store unique prefixes
unique_prefixes = defaultdict(set)

# Extract unique prefixes
for filename in files:
    # Split filename by underscore and get the first part (e.g. "compound-003" or "gene-002")
    prefix = filename.split('_')[0]
    
    # Determine if it's a compound or gene and store
    if prefix.startswith('compound'):
        unique_prefixes['compounds'].add(prefix)
    elif prefix.startswith('gene'):
        unique_prefixes['genes'].add(prefix)

# Print results
print(f"Number of unique compounds: {len(unique_prefixes['compounds'])}")
print(f"Number of unique genes: {len(unique_prefixes['genes'])}")

print("\nUnique compounds:")
for compound in sorted(unique_prefixes['compounds']):
    print(compound)

print("\nUnique genes:")
for gene in sorted(unique_prefixes['genes']):
    print(gene)


Number of unique compounds: 1
Number of unique genes: 1

Unique compounds:
compound-001

Unique genes:
gene-001


In [43]:
# Dictionary to store counts for each prefix
prefix_counts = defaultdict(int)

# Count occurrences of each prefix
for filename in files:
    prefix = filename.split('_')[0]
    prefix_counts[prefix] += 1

print("\nCounts for each compound:")
for compound in sorted(unique_prefixes['compounds']):
    print(f"{compound}: {prefix_counts[compound]} images")

print("\nCounts for each gene:")
for gene in sorted(unique_prefixes['genes']):
    print(f"{gene}: {prefix_counts[gene]} images")



Counts for each compound:
compound-001: 32886 images

Counts for each gene:
gene-001: 32886 images


In [37]:
import os
import random
from pathlib import Path

# Set the data directory
data_dir = "/Volumes/External SSD/cp_1_gene_1_mixed/"

# First delete compound-002 and compound-003 images
deleted_count = 0
for filename in os.listdir(data_dir):
    if filename.startswith(('compound-002', 'compound-003')):
        os.remove(os.path.join(data_dir, filename))
        deleted_count += 1
print(f"Deleted {deleted_count} images for compound-002 and compound-003")

# Get remaining compound-001 and gene-001 files
compound_001_files = []
gene_001_files = []
for filename in os.listdir(data_dir):
    if filename.startswith('compound-001'):
        compound_001_files.append(filename)
    elif filename.startswith('gene-001'):
        gene_001_files.append(filename)

print(f"\nBefore balancing:")
print(f"compound-001: {len(compound_001_files)} images")
print(f"gene-001: {len(gene_001_files)} images")

# Balance datasets by randomly removing compound-001 images
if len(compound_001_files) > len(gene_001_files):
    files_to_remove = random.sample(compound_001_files, len(compound_001_files) - len(gene_001_files))
    for filename in files_to_remove:
        os.remove(os.path.join(data_dir, filename))
    print(f"\nRemoved {len(files_to_remove)} compound-001 images to balance datasets")

# Verify final counts
final_compound = len([f for f in os.listdir(data_dir) if f.startswith('compound-001')])
final_gene = len([f for f in os.listdir(data_dir) if f.startswith('gene-001')])

print(f"\nAfter balancing:")
print(f"compound-001: {final_compound} images")
print(f"gene-001: {final_gene} images")


Deleted 6912 images for compound-002 and compound-003

Before balancing:
compound-001: 59484 images
gene-001: 32886 images

Removed 26598 compound-001 images to balance datasets

After balancing:
compound-001: 32886 images
gene-001: 32886 images
